In [ ]:
import os
import pickle

from astropy.coordinates import SkyCoord
from astropy.time import Time
from astropy import units as u
import astropy_healpix as ah
import ligo.skymap.plot
import ligo.skymap.io
import ligo.skymap.postprocess
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from thesis_utils.plotting import save_figure
from utils import ANALYSIS_LABELS, EVENTS

# set_plotting()
# Breaks formatting for skymaps, so set manually
sns.set_palette("colorblind")
plt.rcParams["figure.figsize"] = (6.141732 / 1.5, 3.79579912579 / 1.5)
plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"

In [ ]:
base_path = "skymaps"
analyses = ["GWTC-2", "nessai-XPHM"]
colours = ["C0", "C1"]

In [ ]:
# Based on https://git.ligo.org/lscsoft/ligo.skymap/-/blob/main/ligo/skymap/tool/ligo_skymap_plot.pyb

for event in EVENTS:
    fig = plt.figure()
    ax = plt.axes(
        projection="astro degrees mollweide",
        # center=SkyCoord("150deg -70deg"),
        # radius="20deg",
    )
    ax.grid()

    contours = [50, 90]
    text = []
    lw = 0.8

    for analysis, colour in zip(analyses, colours):
        fits_file = os.path.join(base_path, event, analysis, "skymap.fits")

        skymap, metadata = ligo.skymap.io.fits.read_sky_map(fits_file)
        nside = ah.npix_to_nside(len(skymap))
        deg2perpix = ah.nside_to_pixel_area(nside).to_value(u.deg**2)
        probperdeg2 = skymap / deg2perpix

        cls = 100 * ligo.skymap.postprocess.find_greedy_credible_levels(skymap)
        cs = ax.contour_hpx(
            (cls, "ICRS"),
            # nested=metadata['nest'],
            colors=colour,
            linewidths=lw,
            levels=contours,
        )
        pp = np.round(contours).astype(int)
        ii = np.round(np.searchsorted(np.sort(cls), contours) * deg2perpix).astype(int)
        text_areas = []
        for i, p in zip(ii, pp):
            text_areas.append(f"{p:d}\%: {i:,d}" + r"$\;\textrm{deg}^{2}$")
        text.append(f"{ANALYSIS_LABELS.get(analysis)}: {', '.join(text_areas)}")

    handle_elements = [
        Line2D([0], [0], color="C0", label=text[0], lw=lw),
        Line2D([0], [0], color="C1", label=text[1], lw=lw),
    ]

    # ax.text(1, -0.15, '\n'.join(text), transform=ax.transAxes, ha='right')
    # ax.text(0, 1, event, transform=ax.transAxes, ha='left', fontsize=14)
    plt.legend(
        handles=handle_elements,
        frameon=False,
        # loc="lower right",
        bbox_to_anchor=(1.1, 0.0),
    )
    plt.show()
    save_figure(fig, f"{event}_skymap", "figures/skymaps/", bbox_inches="tight")